# Import

In [ ]:
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split, ShuffleSplit, learning_curve, cross_val_score
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import log_loss, confusion_matrix, classification_report, roc_curve, auc, accuracy_score
from sklearn.preprocessing import label_binarize

import Dimension_Reduce_Draw as DRD
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP

In [ ]:
file = './合併_6.csv'

'''
Target = 0 : 反應前
Target = 1 : 加 acetone
Target = 2 : 加 octanal
Target = 3 : 加 heptanal
'''

'''validation 的 AUC = 0.96，test 的 AUC = 0.95，Average XGBscore: 0.81'''

In [ ]:
%%time

np.random.seed(0)

################################################### 全部特徵 ###################################################
data = pd.read_csv(file)
data['Index'] = list(range(data.shape[0]))
data.set_index('Index', inplace = True)
n_columns = data.shape[1] - 1
feature = data.iloc[:, 1:n_columns]
target = data.iloc[:, n_columns]
n_class = target.unique().size

################################################### 抓特徵(相關係數) ###################################################
corrs = []
for c in feature.columns:
    corr = np.corrcoef(feature[c], target)[0, 1]
    corrs.append(corr)
corrs = np.array(corrs)
index = np.argsort(np.abs(corrs))[::-1]
top_cols, top_importance = feature.columns.values[index][:300], corrs[index][:300]

################################################### 新特徵並標準化 ###################################################
feature = data.loc[:, top_cols]
feature = StandardScaler().fit(feature).transform(feature) 
feature = pd.DataFrame(feature)

################################################### optuna ###################################################
def objective(trial):
    params = {'objective': 'multi:softmax',
              'eval_metric': 'mlogloss',
              'n_estimators': trial.suggest_int('n_estimators', 100, 300),
              'max_depth': trial.suggest_int('max_depth', 6, 18),
              'eta': trial.suggest_loguniform('eta', 1e-4, 1.0),
              'random_state': trial.suggest_int('random_state', 1999, 2023),
              'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
              'gamma': trial.suggest_loguniform('gamma', 1e-5, 10),
              'subsample': trial.suggest_float('subsample', 0.001, 1.0),
              'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 1e-3, 1),
              'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
              'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
              'use_label_encoder': False,
              'num_class': 4,
              'seed': 42
              }  
    scores = []
    kf = KFold(n_splits = 5, shuffle = True, random_state = 0)
    for train_index, valid_index in kf.split(feature):
        X_train, X_valid = feature.iloc[train_index], feature.iloc[valid_index]
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
        model = XGBClassifier(**params)
        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 10, verbose = False)
        y_pred = model.predict_proba(X_valid)
        score = log_loss(y_valid, y_pred)
        scores.append(score)
    mean_score = np.mean(scores)
    return mean_score
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(sampler = optuna.samplers.RandomSampler(seed = 0), direction = 'minimize', study_name = 'lr', pruner = optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials = 50)
the_best_params = study.best_params

################################################### 分訓練組、驗證組及測試組 ###################################################
X_train, X_valid, y_train, y_valid = train_test_split(feature, target, test_size = 0.4, random_state = 0, stratify = target)
X_test, X_valid, y_test, y_valid = train_test_split(X_valid, y_valid, test_size = 0.5, random_state = 0, stratify = y_valid)     

################################################### 建模並預測 ###################################################
model_XGB = XGBClassifier(**the_best_params)
model_XGB.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 10, verbose = False)
y_pred_valid = model_XGB.predict_proba(X_valid)
y_pred_valid = [np.argmax(line) for line in y_pred_valid]
y_pred_test = model_XGB.predict_proba(X_test) 
y_pred_test = [np.argmax(line) for line in y_pred_test]

################################################### 學習曲線 ###################################################
cv = ShuffleSplit(n_splits = 100, test_size = 0.2, random_state = 0) # 定義學習曲線的訓練集大小
train_sizes, train_scores, test_scores = learning_curve(model_XGB, X_train, y_train, cv = cv, n_jobs = -1, train_sizes = np.linspace(.1, 1.0, 5), scoring = "accuracy") # 繪製學習曲線 
train_mean = np.mean(train_scores, axis = 1) # 計算平均分數和標準差
train_std = np.std(train_scores, axis = 1)
test_mean = np.mean(test_scores, axis = 1)
test_std = np.std(test_scores, axis = 1)
plt.style.use('ggplot')
plt.title('Learning Curve')
plt.plot(train_sizes, train_mean, 'o-', color = "r", label = "Training score")
plt.plot(train_sizes, test_mean, 'o-', color = "g", label = "Cross-validation score")
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha = 0.1, color = "r")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha = 0.1, color = "g")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
plt.legend(loc = "best")
plt.show()

################################################### Validation 混淆矩陣 ###################################################
labels = np.unique(y_valid)
cm = confusion_matrix(y_valid, y_pred_valid, labels = labels)
cm_df = pd.DataFrame(cm)
plt.figure(figsize = (5, 4))
sns.heatmap(cm_df, annot = True, cmap = 'summer')
plt.title('Confusion Matrix Validation')
plt.xlabel('Predicted Values')
plt.ylabel('Actal Values')
plt.show()

################################################### Test 混淆矩陣 ###################################################
labels = np.unique(y_test)
cm = confusion_matrix(y_test, y_pred_test, labels = labels)
cm_df = pd.DataFrame(cm)
plt.figure(figsize = (5, 4))
sns.heatmap(cm_df, annot = True, cmap = 'summer')
plt.title('Confusion Matrix Test')
plt.xlabel('Predicted Values')
plt.ylabel('Actal Values')
plt.show()

################################################### 評價模型 ###################################################
print('訓練集: ', model_XGB.score(X_train, y_train))
print('驗證集: ', model_XGB.score(X_valid, y_valid))
print(classification_report(y_valid, y_pred_valid))
print('測試集: ', model_XGB.score(X_test, y_test))
print(classification_report(y_test, y_pred_test))


################################################### Validation AUC ###################################################
y_valid = label_binarize(y_valid, np.arange(n_class))
y_pred_valid = model_XGB.predict_proba(X_valid)
fpr, tpr, threshold = roc_curve(y_valid.ravel(), y_pred_valid.ravel())
auc_1 = auc(fpr, tpr)
plt.title('Validation AUC')
plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc_1)
plt.plot([0, 1], [0, 1], c = '#808080', lw = 1, ls = '--', alpha = 0.7)
plt.legend(loc = 'lower right', fancybox = True, framealpha = 0.8, fontsize = 12)
plt.xlim((-0.01, 1.02))
plt.ylim((-0.01, 1.02))
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('False Positive Rate', fontsize = 13)
plt.ylabel('True Positive Rate', fontsize = 13)
plt.grid(b = True, ls = ':')
plt.show()

################################################### Test AUC ###################################################
y_test = label_binarize(y_test, np.arange(n_class))
y_pred_test = model_XGB.predict_proba(X_test)
fpr, tpr, threshold = roc_curve(y_test.ravel(), y_pred_test.ravel())
auc_1 = auc(fpr, tpr)
plt.title('Test AUC')
plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc_1)
plt.plot([0, 1], [0, 1], c = '#808080', lw = 1, ls = '--', alpha = 0.7)
plt.legend(loc = 'lower right', fancybox = True, framealpha = 0.8, fontsize = 12)
plt.xlim((-0.01, 1.02))
plt.ylim((-0.01, 1.02))
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('False Positive Rate', fontsize = 13)
plt.ylabel('True Positive Rate', fontsize = 13)
plt.grid(b = True, ls = ':')
plt.show()

################################################### 特徵重要性 ###################################################
plot_importance(model_XGB, max_num_features = 30)
plt.show()

################################################### 交叉驗證後的正確率 ###################################################
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)
XGBscore = cross_val_score(XGBClassifier(**the_best_params), feature, target, cv = kf, scoring = "accuracy")
print(f'XGBscore for each fold are: {XGBscore}')
print(f'Average XGBscore: {"{:.2f}".format(XGBscore.mean())}')

# [Importance](https://machinelearningmastery.com/calculate-feature-importance-with-python/)

In [ ]:
DRD.Draw_PCA_Multi(file)
DRD.Draw_t_SNE_Multi(file)
DRD.Draw_UMAP_Multi(file)

In [ ]:
np.random.seed(0)

################################################### 全部特徵 ###################################################
data = pd.read_csv(file)
data['Index'] = list(range(data.shape[0]))
data.set_index('Index', inplace = True)
n_columns = data.shape[1] - 1
feature = data.iloc[:, 1:n_columns]
target = data.iloc[:, n_columns]

################################################### 抓特徵(相關係數) ###################################################
corrs = []
for c in feature.columns:
    corr = np.corrcoef(feature[c], target)[0, 1]
    corrs.append(corr)
corrs = np.array(corrs)
index = np.argsort(np.abs(corrs))[::-1]
top_cols, top_importance = feature.columns.values[index][:300], corrs[index][:300]

################################################### 新特徵並標準化 ###################################################
feature = data.loc[:, top_cols]
feature = StandardScaler().fit(feature).transform(feature) 
feature = pd.DataFrame(feature)
data = pd.merge(feature, target, left_index = True, right_index = True)

################################################### PCA ###################################################
n_columns = data.shape[1] - 1
feature = data.iloc[:, :n_columns]
target = data.iloc[:, n_columns]
pca = PCA(n_components = 3).fit(feature).transform(feature)
Xax = pca[:, 0]
Yax = pca[:, 1]
Zax = pca[:, 2]
color = {0:'red', 1:'skyblue', 2:'green', 3:'purple'}
label = {0:'before', 1:'acetone', 2:'octanal', 3:'heptanal'}
marker = {0:'*', 1:'*', 2:'*', 3:'*'}
alpha = {0:.3, 1:.5, 2:.3, 3:.5}
fig = plt.figure(figsize = (12, 8))
ax = fig.add_subplot(111, projection = '3d')
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_zlabel('Principal Component 3', fontsize = 15)
ax.set_title('acetone_51', fontsize = 15)
fig.patch.set_facecolor('white')
for i in np.unique(target):
    ix = np.where(target == i)
    ax.scatter(Xax[ix], Yax[ix], Zax[ix], c = color[i], s = 40, label = label[i], marker = marker[i], alpha = alpha[i])
ax.grid()
plt.show()
df = pd.DataFrame(pca, columns = ('x', 'y', 'z'))
df["class"] = target
df["class"] = df["class"].replace({0:'before', 1:'acetone', 2:'octanal', 3:'heptanal'})
pca_interactive = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = 'class', title = 'acetone_51')
pca_interactive.update_traces(marker_size = 5)
pca_interactive.show()  

################################################### t_SNE ###################################################
n_columns = data.shape[1] - 1
feature = data.iloc[:, :n_columns]
target = data.iloc[:, n_columns]
t_SNE = TSNE(n_components = 2, init = 'random', random_state = 5, verbose = 1).fit_transform(feature)
x_min, x_max = t_SNE.min(0), t_SNE.max(0)
X_norm = (t_SNE - x_min) / (x_max - x_min)
plt.figure(figsize = (5, 5))
for i in range(X_norm.shape[0]):
    plt.text(X_norm[i, 0], X_norm[i, 1], str(target[i]), color = plt.cm.Set1(target[i]), fontdict = {'weight': 'bold', 'size': 9})
plt.title('acetone_51', fontsize = 15)   
plt.xticks([])
plt.yticks([])
plt.show()
t_SNE = TSNE(n_components = 3, init = 'random', random_state = 5, verbose = 1).fit_transform(feature)
df = pd.DataFrame(t_SNE, columns = ('x', 'y', 'z'))
df["class"] = target
df["class"] = df["class"].replace({0:'before', 1:'acetone', 2:'octanal', 3:'heptanal'})
t_SNE_interactive = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = 'class', title = 'acetone_51')
t_SNE_interactive.update_traces(marker_size = 5)
t_SNE_interactive.show()

################################################### UMAP ###################################################
n_columns = data.shape[1] - 1
feature = data.iloc[:, :n_columns]
target = data.iloc[:, n_columns]
embedding = UMAP(random_state = 42).fit_transform(feature)
df = pd.DataFrame(embedding, columns = ('x', 'y'))
df["class"] = target
df["class"] = df["class"].replace({0:'before', 1:'acetone', 2:'octanal', 3:'heptanal'})
sns.set_style("whitegrid", {'axes.grid' : True})
ax = sns.pairplot(x_vars = ["x"], y_vars = ["y"], data = df, hue = "class", size = 5, plot_kws = {"s": 20})
ax.fig.suptitle('acetone_51', fontsize = 15)
plt.show()
proj_3d = UMAP(n_components = 3, init = 'random', random_state = 0).fit_transform(feature)
df = pd.DataFrame(proj_3d, columns = ('x', 'y', 'z'))
df["class"] = target
df["class"] = df["class"].replace({0:'before', 1:'acetone', 2:'octanal', 3:'heptanal'})
UMAP_interactive = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = 'class', title = 'acetone_51')
UMAP_interactive.update_traces(marker_size = 5)
UMAP_interactive.show()

In [ ]:
train_file = './all_6.csv'
test_file = './all_6_new.csv'

'''
all_6 預測 validation 的 AUC = 0.98
all_6 預測 all_6_new(test) 的 AUC = 0.80

Average XGBscore: 0.86
'''

'''
all_6_new 預測 validation 的 AUC = 0.92
all_6_new 預測 all_6(test) 的 AUC = 0.80

Average XGBscore: 0.86
'''

In [ ]:
%%time

np.random.seed(0)

################################################### 全部特徵 ###################################################
data = pd.read_csv(train_file)
data['Index'] = list(range(data.shape[0]))
data.set_index('Index', inplace = True)
n_columns = data.shape[1] - 1
feature = data.iloc[:, 1:n_columns]
target = data.iloc[:, n_columns]
n_class = target.unique().size

################################################### 抓特徵(相關係數) ###################################################
corrs = []
for c in feature.columns:
    corr = np.corrcoef(feature[c], target)[0, 1]
    corrs.append(corr)
corrs = np.array(corrs)
index = np.argsort(np.abs(corrs))[::-1]
top_cols, top_importance = feature.columns.values[index][:100], corrs[index][:100]

################################################### 新特徵並標準化 ###################################################
feature = data.loc[:, top_cols]
feature = StandardScaler().fit(feature).transform(feature) 
feature = pd.DataFrame(feature)

################################################### optuna ###################################################
def objective(trial):
    params = {'objective': 'multi:softmax',
              'eval_metric': 'mlogloss',
              'n_estimators': trial.suggest_int('n_estimators', 50, 150),
              'max_depth': trial.suggest_int('max_depth', 6, 18),
              'eta': trial.suggest_loguniform('eta', 1e-3, 1.0),
              'random_state': trial.suggest_int('random_state', 1999, 2023),
              'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
              'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
              'subsample': trial.suggest_loguniform('subsample', 0.1, 1.0),
              'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
              'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
              'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
              'use_label_encoder': False,
              'num_class': 4,
              'seed': 42
              }  
    scores = []
    kf = KFold(n_splits = 5, shuffle = True, random_state = 0)
    for train_index, valid_index in kf.split(feature):
        X_train, X_valid = feature.iloc[train_index], feature.iloc[valid_index]
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
        model = XGBClassifier(**params)
        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 10, verbose = False)
        y_pred = model.predict_proba(X_valid)
        score = log_loss(y_valid, y_pred)
        scores.append(score)
    mean_score = np.mean(scores)
    return mean_score
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(sampler = optuna.samplers.RandomSampler(seed = 0), direction = 'minimize', study_name = 'lr', pruner = optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials = 50)
the_best_params = study.best_params

################################################### 分訓練組、驗證組及測試組 ###################################################
X_train, X_valid, y_train, y_valid = train_test_split(feature, target, test_size = 0.4, random_state = 0, stratify = target)
test = pd.read_csv(test_file)
test['Index'] = list(range(test.shape[0]))
test.set_index('Index', inplace = True)
n_columns = test.shape[1] - 1
X_test = test.loc[:, top_cols]
y_test = test.iloc[:, n_columns]
X_test = StandardScaler().fit(X_test).transform(X_test) 
X_test = pd.DataFrame(X_test)
X_test, _, y_test, _ = train_test_split(X_test, y_test, test_size = 0.6, random_state = 0, stratify = y_test)

################################################### 建模並預測 ###################################################
model_XGB = XGBClassifier(**the_best_params)
model_XGB.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 10, verbose = False)
y_pred_valid = model_XGB.predict_proba(X_valid)
y_pred_valid = [np.argmax(line) for line in y_pred_valid]
y_pred_test = model_XGB.predict_proba(X_test) 
y_pred_test = [np.argmax(line) for line in y_pred_test]

################################################### 學習曲線 ###################################################
cv = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 0) # 定義學習曲線的訓練集大小
train_sizes, train_scores, test_scores = learning_curve(model_XGB, X_train, y_train, cv = cv, n_jobs = -1, train_sizes = np.linspace(.1, 1.0, 5), scoring = "accuracy") # 繪製學習曲線 
train_mean = np.mean(train_scores, axis = 1) # 計算平均分數和標準差
train_std = np.std(train_scores, axis = 1)
test_mean = np.mean(test_scores, axis = 1)
test_std = np.std(test_scores, axis = 1)
plt.title('Learning Curve')
plt.plot(train_sizes, train_mean, 'o-', color = "r", label = "Training score")
plt.plot(train_sizes, test_mean, 'o-', color = "g", label = "Cross-validation score")
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha = 0.1, color = "r")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha = 0.1, color = "g")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
plt.legend(loc = "best")
plt.show()

################################################### Validation 混淆矩陣 ###################################################
labels = np.unique(y_valid)
cm = confusion_matrix(y_valid, y_pred_valid, labels = labels)
cm_df = pd.DataFrame(cm)
plt.figure(figsize = (5, 4))
sns.heatmap(cm_df, annot = True, cmap = 'summer')
plt.title('Confusion Matrix Validation')
plt.xlabel('Predicted Values')
plt.ylabel('Actal Values')
plt.show()

################################################### Test 混淆矩陣 ###################################################
labels = np.unique(y_test)
cm = confusion_matrix(y_test, y_pred_test, labels = labels)
cm_df = pd.DataFrame(cm)
plt.figure(figsize = (5, 4))
sns.heatmap(cm_df, annot = True, cmap = 'summer')
plt.title('Confusion Matrix Test')
plt.xlabel('Predicted Values')
plt.ylabel('Actal Values')
plt.show()

################################################### 評價模型 ###################################################
print('訓練集: ', model_XGB.score(X_train, y_train))
print('驗證集: ', model_XGB.score(X_valid, y_valid))
print(classification_report(y_valid, y_pred_valid))
print('測試集: ', model_XGB.score(X_test, y_test))
print(classification_report(y_test, y_pred_test))

################################################### Validation AUC ###################################################
y_valid = label_binarize(y_valid, np.arange(n_class))
y_pred_valid = model_XGB.predict_proba(X_valid)
fpr, tpr, threshold = roc_curve(y_valid.ravel(), y_pred_valid.ravel())
auc_1 = auc(fpr, tpr)
plt.title('Validation AUC')
plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc_1)
plt.plot([0, 1], [0, 1], c = '#808080', lw = 1, ls = '--', alpha = 0.7)
plt.legend(loc = 'lower right', fancybox = True, framealpha = 0.8, fontsize = 12)
plt.xlim((-0.01, 1.02))
plt.ylim((-0.01, 1.02))
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('False Positive Rate', fontsize = 13)
plt.ylabel('True Positive Rate', fontsize = 13)
plt.grid(b = True, ls = ':')
plt.show()

################################################### Test AUC ###################################################
y_test = label_binarize(y_test, np.arange(n_class))
y_pred_test = model_XGB.predict_proba(X_test)
fpr, tpr, threshold = roc_curve(y_test.ravel(), y_pred_test.ravel())
auc_1 = auc(fpr, tpr)
plt.title('Test AUC')
plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc_1)
plt.plot([0, 1], [0, 1], c = '#808080', lw = 1, ls = '--', alpha = 0.7)
plt.legend(loc = 'lower right', fancybox = True, framealpha = 0.8, fontsize = 12)
plt.xlim((-0.01, 1.02))
plt.ylim((-0.01, 1.02))
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('False Positive Rate', fontsize = 13)
plt.ylabel('True Positive Rate', fontsize = 13)
plt.grid(b = True, ls = ':')
plt.show()

################################################### 特徵重要性 ###################################################
plot_importance(model_XGB, max_num_features = 30)
plt.show()

kf = KFold(n_splits = 5, shuffle = True, random_state = 0)
XGBscore = cross_val_score(XGBClassifier(**the_best_params), feature, target, cv = kf, scoring = "accuracy")
print(f'XGBscore for each fold are: {XGBscore}')
print(f'Average XGBscore: {"{:.2f}".format(XGBscore.mean())}')